In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
# Define the custom activation function
def identity_activation(x):
    return tf.identity(x)

# Load the model with the custom_objects argument
loaded_model_1 = tf.keras.models.load_model("my_model_1.h5", custom_objects={"identity_activation": identity_activation})
loaded_model_2 = tf.keras.models.load_model("my_model_2.h5", custom_objects={"identity_activation": identity_activation})

In [2]:
df = pd.read_excel('combined_data.xlsx')
# Drop the extra index column
df = df.drop(columns=['Unnamed: 0']) 
df

,E (GPa),σu (MPa),σf (MPa),RA (%),εf,HB,σL (MPa)
0,197,783.2,1207.0,55.1,0.800,180,260.0
1,196,905.5,1247.0,41.7,0.540,195,311.7
2,211,821.0,1128.0,49.6,0.700,238,254.5
3,202,828.5,1065.0,34.1,0.416,220,360.2
4,220,2360.0,2115.0,14.7,0.160,536,796.1
...,...,...,...,...,...,...,...
166,205,565.0,931.0,64.0,1.030,159,205.1
167,205,565.0,1000.0,69.0,1.190,150,196.6
168,205,440.0,752.0,65.0,1.060,150,210.5
169,205,530.0,1000.0,72.0,1.240,156,235.5


In [3]:

# Adding columns for comparision metrics
# For Model 1
df['log_σu'] = df['σu (MPa)'].apply(lambda x: np.log(x))
df['log_σf'] = df['σf (MPa)'].apply(lambda x: np.log(x))

# For Model 2
df['log_hb'] = df['HB'].apply(lambda x: np.log(x))
df['square_ra'] = df['RA (%)'].apply(lambda x: x ** 2)

# Rearranging the columns
df = df[['E (GPa)', 'σu (MPa)','log_σu','σf (MPa)', 'log_σf' , 'εf' , 'RA (%)', 'square_ra', 'HB' , 'log_hb',  'σL (MPa)']
       ]
df

,E (GPa),σu (MPa),log_σu,σf (MPa),log_σf,εf,RA (%),square_ra,HB,log_hb,σL (MPa)
0,197,783.2,6.663388,1207.0,7.095893,0.800,55.1,3036.01,180,5.192957,260.0
1,196,905.5,6.808487,1247.0,7.128496,0.540,41.7,1738.89,195,5.273000,311.7
2,211,821.0,6.710523,1128.0,7.028201,0.700,49.6,2460.16,238,5.472271,254.5
3,202,828.5,6.719617,1065.0,6.970730,0.416,34.1,1162.81,220,5.393628,360.2
4,220,2360.0,7.766417,2115.0,7.656810,0.160,14.7,216.09,536,6.284134,796.1
...,...,...,...,...,...,...,...,...,...,...,...
166,205,565.0,6.336826,931.0,6.836259,1.030,64.0,4096.00,159,5.068904,205.1
167,205,565.0,6.336826,1000.0,6.907755,1.190,69.0,4761.00,150,5.010635,196.6
168,205,440.0,6.086775,752.0,6.622736,1.060,65.0,4225.00,150,5.010635,210.5
169,205,530.0,6.272877,1000.0,6.907755,1.240,72.0,5184.00,156,5.049856,235.5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   E (GPa)    171 non-null    int64  
 1   σu (MPa)   171 non-null    float64
 2   log_σu     171 non-null    float64
 3   σf (MPa)   171 non-null    float64
 4   log_σf     171 non-null    float64
 5   εf         171 non-null    float64
 6   RA (%)     171 non-null    float64
 7   square_ra  171 non-null    float64
 8   HB         171 non-null    int64  
 9   log_hb     171 non-null    float64
 10  σL (MPa)   171 non-null    float64
dtypes: float64(9), int64(2)
memory usage: 14.8 KB


In [5]:
#Based on FPCM and USM

def evaluate_sigmaF(f, u):
    return 1.12 * f * ((f/u)** 0.893)
def evaluate_b(f, u):
    return (-1 * 0.0792) - 0.179 * math.log10(f/u)

def evaluate_fatigueLimit_predicted_FPCM(coeff , exp , cycles):
    ans = coeff * (cycles ** exp)
    return ans

def evaluate_fatigueLimit_predicted_USM(u, cycles):
    coeff = 1.9018 * u
    exp = -0.12
    ans = coeff * (cycles ** exp)
    return ans


def prediction_via_FPCM(row):
    # Extract values from the row
    f = row['σf (MPa)']
    u = row['σu (MPa)']
    
    # Perform some operation based on the values
    coeff = evaluate_sigmaF(f,u)
    exp = evaluate_b(f,u)
    
    #Defining the cycles
    cycles = 10**6
    result = evaluate_fatigueLimit_predicted_FPCM(coeff , exp, cycles)
    
    return result

# Define your custom function2
def prediction_via_USM(row):
    # Extract values from the row
    f = row['σf (MPa)']
    u = row['σu (MPa)']
    
    #Defining the cycles
    cycles = 10**6
    result = evaluate_fatigueLimit_predicted_USM(u , cycles)
    
    return result


In [6]:
import math
from sklearn.preprocessing import StandardScaler
# Predicting values via different methods
#Conventional Methods
df['Fatigue_FPCM'] = df.apply(prediction_via_FPCM, axis=1)  # Apply along rows
df['Fatigue_USM'] = df.apply(prediction_via_USM, axis=1)  # Apply along rows

# Load your trained StandardScaler for each model




#Neural Network model-1
df_model_1 = df[['log_σu', 'log_σf', 'εf']]

scaler_model_1 = StandardScaler()
scaler_model_1 = scaler_model_1.fit(df_model_1)  # X_train_model_1 should be the data used to train model 1

scaled_df_model_1 = pd.DataFrame(scaler_model_1.transform(df_model_1), columns=df_model_1.columns)
predictions_1 = loaded_model_1.predict(scaled_df_model_1)
df['Model-1'] =  predictions_1

#Neural Network model-2
df_model_2 = df[['log_hb', 'RA (%)', 'square_ra']]

scaler_model_2 = StandardScaler()
scaler_model_2 = scaler_model_2.fit(df_model_2)  # X_train_model_2 should be the data used to train model 2

scaled_df_model_2 = pd.DataFrame(scaler_model_2.transform(df_model_2), columns=df_model_2.columns)
predictions_2 = loaded_model_2.predict(scaled_df_model_2)
df['Model-2'] =  predictions_2



print("DataFrame with new columns:")
df

C:\Users\adars\AppData\Local\Temp\ipykernel_14936\1236655206.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fatigue_FPCM'] = df.apply(prediction_via_FPCM, axis=1)  # Apply along rows
C:\Users\adars\AppData\Local\Temp\ipykernel_14936\1236655206.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fatigue_USM'] = df.apply(prediction_via_USM, axis=1)  # Apply along rows


6/6 [==============================] - 0s 997us/step
DataFrame with new columns:


C:\Users\adars\AppData\Local\Temp\ipykernel_14936\1236655206.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Model-1'] =  predictions_1
C:\Users\adars\AppData\Local\Temp\ipykernel_14936\1236655206.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Model-2'] =  predictions_2


,E (GPa),σu (MPa),log_σu,σf (MPa),log_σf,εf,RA (%),square_ra,HB,log_hb,σL (MPa),Fatigue_FPCM,Fatigue_USM,Model-1,Model-2
0,197,783.2,6.663388,1207.0,7.095893,0.800,55.1,3036.01,180,5.192957,260.0,418.530615,283.816423,332.196594,199.840897
1,196,905.5,6.808487,1247.0,7.128496,0.540,41.7,1738.89,195,5.273000,311.7,441.295459,328.135560,366.369843,215.521378
2,211,821.0,6.710523,1128.0,7.028201,0.700,49.6,2460.16,238,5.472271,254.5,399.351469,297.514406,334.844269,338.084320
3,202,828.5,6.719617,1065.0,6.970730,0.416,34.1,1162.81,220,5.393628,360.2,381.617512,300.232260,324.464752,268.717560
4,220,2360.0,7.766417,2115.0,7.656810,0.160,14.7,216.09,536,6.284134,796.1,808.992117,855.218026,750.249207,715.424805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,205,565.0,6.336826,931.0,6.836259,1.030,64.0,4096.00,159,5.068904,205.1,318.939699,204.744993,216.645081,212.569931
167,205,565.0,6.336826,1000.0,6.907755,1.190,69.0,4761.00,150,5.010635,196.6,338.172892,204.744993,249.358856,239.141953
168,205,440.0,6.086775,752.0,6.622736,1.060,65.0,4225.00,150,5.010635,210.5,255.920673,159.447429,169.621948,203.313812
169,205,530.0,6.272877,1000.0,6.907755,1.240,72.0,5184.00,156,5.049856,235.5,334.281202,192.061675,249.189545,278.929413


In [7]:
# Calculate absolute differences for each method
df['abs_diff_model1'] = abs(df['σL (MPa)'] - df['Model-1'])
df['abs_diff_model2'] = abs(df['σL (MPa)'] - df['Model-2'])
df['abs_diff_model3'] = abs(df['σL (MPa)'] - df['Fatigue_FPCM'])
df['abs_diff_model4'] = abs(df['σL (MPa)'] - df['Fatigue_USM'])


# Define deviation thresholds
#For Model1
deviation_10_percent_model1 = 0.10 * df['Model-1']
deviation_15_percent_model1 = 0.15 * df['Model-1']
deviation_20_percent_model1 = 0.20 * df['Model-1']

#For Model2
deviation_10_percent_model2 = 0.10 * df['Model-2']
deviation_15_percent_model2 = 0.15 * df['Model-2']
deviation_20_percent_model2 = 0.20 * df['Model-2']

#For Model3
deviation_10_percent_model3 = 0.10 * df['Fatigue_FPCM']
deviation_15_percent_model3 = 0.15 * df['Fatigue_FPCM']
deviation_20_percent_model3 = 0.20 * df['Fatigue_FPCM']


#For Model4
deviation_10_percent_model4 = 0.10 * df['Fatigue_USM']
deviation_15_percent_model4 = 0.15 * df['Fatigue_USM']
deviation_20_percent_model4 = 0.20 * df['Fatigue_USM']


# Calculate the percentage of values within the specified deviations for each method
df['within_10_percent_model1'] = ((df['abs_diff_model1'] <= deviation_10_percent_model1) * 100).astype(int)
df['within_15_percent_model1'] = ((df['abs_diff_model1'] <= deviation_15_percent_model1) * 100).astype(int)
df['within_20_percent_model1'] = ((df['abs_diff_model1'] <= deviation_20_percent_model1) * 100).astype(int)

df['within_10_percent_model2'] = ((df['abs_diff_model2'] <= deviation_10_percent_model2) * 100).astype(int)
df['within_15_percent_model2'] = ((df['abs_diff_model2'] <= deviation_15_percent_model2) * 100).astype(int)
df['within_20_percent_model2'] = ((df['abs_diff_model2'] <= deviation_20_percent_model2) * 100).astype(int)

df['within_10_percent_model3'] = ((df['abs_diff_model3'] <= deviation_10_percent_model3) * 100).astype(int)
df['within_15_percent_model3'] = ((df['abs_diff_model3'] <= deviation_15_percent_model3) * 100).astype(int)
df['within_20_percent_model3'] = ((df['abs_diff_model3'] <= deviation_20_percent_model3) * 100).astype(int)

df['within_10_percent_model4'] = ((df['abs_diff_model4'] <= deviation_10_percent_model4) * 100).astype(int)
df['within_15_percent_model4'] = ((df['abs_diff_model4'] <= deviation_15_percent_model4) * 100).astype(int)
df['within_20_percent_model4'] = ((df['abs_diff_model4'] <= deviation_20_percent_model4) * 100).astype(int)




# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'Model': ['Model-1', 'Model-2', 'FPCM Model', 'USM Model'],
    'Within 10%': [df['within_10_percent_model1'].mean(), df['within_10_percent_model2'].mean(), df['within_10_percent_model3'].mean(),df['within_10_percent_model4'].mean()],
    'Within 15%': [df['within_15_percent_model1'].mean(), df['within_15_percent_model2'].mean(), df['within_15_percent_model3'].mean(), df['within_15_percent_model4'].mean()],
    'Within 20%': [df['within_20_percent_model1'].mean(), df['within_20_percent_model2'].mean(), df['within_20_percent_model3'].mean(), df['within_20_percent_model4'].mean()],

})

results_df

C:\Users\adars\AppData\Local\Temp\ipykernel_14936\3668076108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['abs_diff_model1'] = abs(df['σL (MPa)'] - df['Model-1'])
C:\Users\adars\AppData\Local\Temp\ipykernel_14936\3668076108.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['abs_diff_model2'] = abs(df['σL (MPa)'] - df['Model-2'])
C:\Users\adars\AppData\Local\Temp\ipykernel_14936\3668076108.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,Model,Within 10%,Within 15%,Within 20%
0,Model-1,53.801170,69.590643,82.456140
1,Model-2,53.216374,69.590643,78.947368
2,FPCM Model,21.637427,33.918129,46.198830
3,USM Model,36.257310,54.970760,67.251462


### Plotting the graph